In [1]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns

from nltk import word_tokenize
%load_ext autoreload
%autoreload 2

In [2]:
from nltk.corpus import stopwords

In [7]:
from common_preprocess import *

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adhocmaster\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df = pd.read_csv("../data/bbc_text_cls.csv")
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [8]:
# step 1. lemmatize all
df["text"] = lemmatize(df["text"])

lemmatizing: 100%|█████████████████████████████████████████████████████████████████████| 2225/2225 [00:40<00:00, 55.19it/s]


In [10]:
df.head()

,text,labels
0,Ad sale boost Time Warner profit Quarterly pro...,business
1,Dollar gain on Greenspan speech The dollar hav...,business
2,Yukos unit buyer face loan claim The owner of ...,business
3,High fuel price hit BA 's profit British Airwa...,business
4,Pernod takeover talk lift Domecq Shares in UK ...,business


In [11]:
# tokens samples
words = word_tokenize(df.iloc[0]['text'])
# words

In [12]:
topWords = getTopWords(
    word_tokenize, 
    df["text"], 
    maxSize = 10,
    stopWords=stopwords.words('english')
)
topWords

removing stop words: 100%|███████████████████████████████████████████████████████████| 179/179 [00:00<00:00, 178970.30it/s]


['say', 'mr', 'year', 'would', 'make', 'also', 'people', 'new', 'one', 'take']

In [13]:
topWords = getTopWords(
    word_tokenize, 
    df["text"], 
    maxSize = 10,
    stopWords = None
)
topWords

counting words: 100%|█████████████████████████████████████████████████████████████████| 2225/2225 [00:04<00:00, 472.48it/s]


['the', 'be', 'to', 'a', 'of', 'and', 'in', 'have', 'it', 'for']

In [ ]:
wordToIndex = buildWordToIndex (
    word_tokenize, 
    df["text"], 
    maxSize = 5000,
    stopWords=stopwords.words('english'),
    lowercase=True,
    
)
# wordToIndex

In [18]:
tf = getTermFreqMatrix(df["text"], wordToIndex, lowercase=True)
assert tf.sum() > 0
tf[:2, :10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
len(df)

In [22]:
print(tf[0].sum(), len(df["text"].iloc[0]))

89.0 2513
